# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

In [5]:
ed.get_contents()

'Webpage Title:\nHome - Edward Donner\nWebpage Contents:\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,\nacquired in 2021\n.\nWe work with groundbreaking, proprietary LLMs verticalized for talent, we’ve\npatented\nour matching model, and our award-winning platfor

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [6]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [8]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [9]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [11]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [13]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [14]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/nanonets/Nanonets-OCR-s',
 '/MiniMaxAI/MiniMax-M1-80k',
 '/tencent/Hunyuan3D-2.1',
 '/echo840/MonkeyOCR',
 '/Menlo/Jan-nano',
 '/models',
 '/spaces/ilcve21/Sparc3D',
 '/spaces/enzostvs/deepsite',
 '/spaces/tencent/Hunyuan3D-2.1',
 '/spaces/MiniMaxAI/MiniMax-M1',
 '/spaces/nvidia/PartPacker',
 '/spaces',
 '/datasets/institutional/institutional-books-1.0',
 '/datasets/EssentialAI/essential-web-v1.0',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/nvidia/Nemotron-Personas',
 '/datasets/lingshu-medical-mllm/ReasonMed',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers',
 '/docs/diffusers',
 '

In [15]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'documentation', 'url': 'https://huggingface.co/docs'}]}

In [16]:
get_links("https://anthropic.com")

{'links': [{'type': 'homepage', 'url': 'https://www.anthropic.com/'},
  {'type': 'about page', 'url': 'https://www.anthropic.com/company'},
  {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'},
  {'type': 'team page', 'url': 'https://www.anthropic.com/team'},
  {'type': 'research page', 'url': 'https://www.anthropic.com/research'},
  {'type': 'news page', 'url': 'https://www.anthropic.com/news'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [17]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [18]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
nanonets/Nanonets-OCR-s
Updated
about 3 hours ago
•
98.5k
•
939
MiniMaxAI/MiniMax-M1-80k
Updated
2 days ago
•
6.32k
•
450
tencent/Hunyuan3D-2.1
Updated
about 13 hours ago
•
16k
•
386
Menlo/Jan-nano
Updated
3 days ago
•
20.4k
•
305
echo840/MonkeyOC

In [21]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [22]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [23]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}, {'type': 'contact page', 'url': 'https://discuss.huggingface.co'}, {'type': 'github page', 'url': 'https://github.com/huggingface'}, {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nnanonets/Nanonets-OCR-s\nUpdated\nabout 14 hours ago\n•\n98.5k\n•\n964\nMiniMaxAI/MiniMax-M1-80k\nUpdated\n3 days ago\n•\n6.32k\n•\n456\ntencent/Hunyuan3D-2.1\nUpdated\nabout 23 hours ago\n•\n16k\n•\n395\nMenlo/Jan-nano\nUpdated\n4 days ago\n•\n20.4k\n•\n316\necho840/MonkeyOCR\nUpdated\n1 day ago\n•\n270\n•\n426\nBrowse 1M+ models\nSpaces\nRunning\n560\n560\nSparc3D\n🏃\nNext-Gen High-Resol

In [24]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [25]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

## The AI Community Building the Future

Welcome to Hugging Face, the leading platform where the machine learning community collaborates on models, datasets, and applications. With over 1 million models and 250,000 datasets, Hugging Face is harnessing the power of AI to drive innovation and promote collaboration.

---

### Company Overview

Hugging Face provides a robust environment for developers, researchers, and enterprises to explore and contribute to cutting-edge machine learning technologies. Our platform is open-source, enabling users to create and share models and datasets centered around diverse applications across text, image, video, and audio modalities.

---

### Key Offerings

- **Models:** Explore a comprehensive library of **1M+ models** tailored for a variety of machine learning tasks.
- **Datasets:** Access and share **250k+ datasets** essential for training robust AI systems.
- **Spaces:** Interact with over **400k applications**, showcasing innovative uses of AI.
- **Compute Solutions:** Deploy models efficiently with our optimized Inference Endpoints, starting at just **$0.60/hour** for GPU computing.

---

### Who We Serve

Hugging Face proudly supports **more than 50,000 organizations**, including industry giants such as:

- **Meta**
- **Amazon**
- **Google**
- **Microsoft**
- **Grammarly**

These organizations utilize our platform for collaborative AI development and research, benefiting from our enterprise-level security and support.

---

### Company Culture

At Hugging Face, our mission is to foster community as we build the future of AI. Our culture is centered on collaboration and innovation, reflecting the inclusive and open-source values we champion:

- **Collaboration:** We believe in the power of community and open sharing of ideas.
- **Innovation:** We're committed to pushing the boundaries of machine learning technologies and applications.
- **Respect & Inclusiveness:** We aim to create a welcoming environment for all who engage with our platform.

---

### Career Opportunities

Join our growing team at Hugging Face! We are always on the lookout for passionate individuals who want to make an impact in the AI community. Careers at Hugging Face offer you the chance to work alongside talented professionals while contributing to transformative technology.

Explore our [Jobs Page](https://huggingface.co/jobs) for current openings and start your journey with us!

---

### Connect With Us

- **GitHub:** [Hugging Face on GitHub](https://github.com/huggingface)
- **Twitter:** [@HuggingFace](https://twitter.com/huggingface)
- **LinkedIn:** [Hugging Face on LinkedIn](https://www.linkedin.com/company/hugging-face)
- **Discord:** [Join Our Community](https://discord.gg/huggingface)

---

Join us at Hugging Face, where we are revolutionizing the landscape of AI through community collaboration, innovation, and education. Together, we are building a future powered by artificial intelligence.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [29]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [32]:
def stream_brochure2(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        #response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [33]:
stream_brochure2("CosidLab", "https://codislabgraz.org")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.tugraz.at/institute/hcc/home'}, {'type': 'research page', 'url': 'https://www.tugraz.at/institute/hcc/research'}, {'type': 'teaching page', 'url': 'https://www.tugraz.at/institute/hcc/teaching'}, {'type': 'careers page', 'url': 'https://codislabgraz.org/theses/'}, {'type': 'social media', 'url': 'https://www.facebook.com/CODIS-Lab-Graz-109137544243601/'}, {'type': 'social media', 'url': 'https://www.instagram.com/codis_lab_graz/'}, {'type': 'social media', 'url': 'https://twitter.com/CoDiSLabGraz'}]}


# CosidLab Brochure

**Welcome to CosidLab**  
_Cognitive & Digital Science Lab at the Institute of Human-Centred Computing (HCC)_

### About Us
At CosidLab, we are dedicated to advancing interdisciplinary research that bridges computer science and cognitive psychology. Our mission is to explore the dynamic interplay between human cognition and digital technologies, fostering innovative solutions that enhance human-computer interactions.  

Our core areas of research include:
- **Digital Literacy Development**
- **Human and Computational Decision Making**
- **Human-Centred System Design and Evaluation**  

Through these themes, we harness expertise in extended reality technologies, engaging media, and data analytics, coupled with insights into human behavior, motivation, and cognition.

### Our Team
CosidLab consists of a passionate team of 6 researchers, 10 PhD students, and various visiting academics who collaboratively push the boundaries of cognitive and digital science. Each member brings unique expertise to our projects, ensuring a rich environment for research and innovation.

### Research Impact
Our lab is engaged in several funded and internal projects aimed at addressing pressing challenges in human-computer interaction and cognitive science. We focus on theoretical and cognitive models, empirical studies, software applications, and evaluations that inform the next generation of digital technologies.

### Company Culture
At CosidLab, we foster a culture of collaboration, creativity, and inclusivity. We believe that the best ideas emerge from diverse perspectives. Our team members thrive in a supportive environment where they can contribute to meaningful research while pursuing their personal and professional development.

### Our Customers
CosidLab collaborates with various stakeholders including academic institutions, industry partners, and governmental organizations. Our work contributes to real-world applications across multiple domains such as education, healthcare, and social sciences, helping organizations leverage technology to meet human needs effectively.

### Career Opportunities
We are always on the lookout for talented individuals who are passionate about the intersection of cognitive science and technology. CosidLab offers an array of career opportunities for researchers, PhD candidates, and students interested in contributing to innovative research projects. Explore open bachelor and master thesis topics and embark on a rewarding journey with us.

### Get in Touch
Join us as we explore the future of human-centred computing.  
For more information, reach out at [CosidLab](http://isds.tugraz.at/codis) or contact us at c.guetl@tugraz.at.

---

**Discover. Innovate. Inspire.**  
_Cognitive & Digital Science Lab (CoDiS Lab), Institute of Human-Centred Computing, Graz University of Technology_

In [31]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Company Brochure

---

### Welcome to Hugging Face

**"The AI community building the future."**

Hugging Face is a collaborative platform at the forefront of machine learning, empowering developers, researchers, and organizations to create, discover, and share models, datasets, and applications. With a commitment to democratizing AI, we offer a vibrant community and cutting-edge tools that enable the next generation of ML innovation.

---

### Our Offerings

- **Models**: Browse over 1 million state-of-the-art machine learning models across a variety of modalities including text, image, video, audio, and 3D.
- **Datasets**: Access a rich repository of more than 250,000 datasets suitable for diverse machine learning tasks.
- **Spaces**: Explore interactive applications in AI, showcasing the latest advancements and breakthroughs in the field.

### Why Choose Hugging Face?

- **Community-Driven**: Join a community of over 50,000 organizations, collaborating and contributing to the evolution of machine learning.
- **Open Source**: With our open-source tools like Transformers, Diffusers, and Tokenizers, we are laying the groundwork for the ML tooling of tomorrow.
- **Enterprise Solutions**: For businesses, we provide enterprise-grade security, dedicated support, and tailored solutions to meet complex needs.

---

### Company Culture

At Hugging Face, we champion collaboration, innovation, and inclusivity. Our diverse and talented team embraces a culture that fosters continuous learning and encourages members to push the boundaries of what's possible in AI.

**Core values:**
- **Democratization of AI**: We believe that machine learning should be accessible to everyone.
- **Innovation**: We're committed to driving the development of cutting-edge technologies that empower our users.
- **Community Engagement**: We actively engage with our users, encouraging feedback and collaboration to enhance our offerings.

---

### Customer Success

Hugging Face is trusted by top-tier organizations, including:

- **Meta**
- **Google**
- **Amazon**
- **Microsoft**
- **Intel**

With our platform, these enterprises deploy advanced AI solutions and harness the power of machine learning to drive their business forward.

---

### Join the Team

Hugging Face is always on the lookout for passionate individuals to join our dynamic team. We offer a range of exciting career opportunities across various domains. 

### Careers at Hugging Face:
- Engineering
- Data Science
- Community Management
- Product Design

**Benefits of working with us:**
- Flexible work environment
- Opportunities for growth and skill development
- A vibrant community of AI enthusiasts

**[Explore Career Opportunities →](https://huggingface.co/jobs)**

---

### Connect with Us

Join our community of machine learning practitioners at Hugging Face. Learn, collaborate, and innovate together!

- **Website**: [huggingface.co](https://huggingface.co)
- **Twitter**: [@huggingface](https://twitter.com/huggingface)
- **Discord**: [Join our Discord Community](https://discord.gg/huggingface)

---

**Hugging Face: Shaping the future of AI, together!**

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>